# Psychometric Classifier of Personality Traits
# Психометрический классификатор черт личности по Майерс-Бриггс
(Psychometric Classifier of Personality Traits(

Этот модуль классифицирует людей по типам личности mbti на основе их последних 50 сообщений в социальных сетях с помощью базового классификатора naivebayesclassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
from nltk.classify import NaiveBayesClassifier

### Импорт датасета (Kaggle)

data_set = pd.read_csv("FFM_1.csv")
data_set.tail()

### Проверка датасетов на наличие отсутствующих значений

data_set.isnull().any()

## Изучение датасета

Размер датасета

data_set.shape

Просмотр постов в поле posts

data_set.iloc[0,1].split('|||')

Определение количества сообщений

len(data_set.iloc[1,1].split('|||'))

Нахождение уникальных значений из колонки типа личности

types = np.unique(np.array(data_set['type']))
types

Общее количество сообщений для каждого типа

total = data_set.groupby(['type']).count()*50
total

Построение графика для лучшей визуализации

plt.figure(figsize = (12,6))

plt.bar(np.array(total.index), height = total['posts'],)
plt.xlabel('Personality types', size = 14)
plt.ylabel('Number of posts available', size = 14)
plt.title('Total posts for each personality type')

## Организация данных для создания модели слов-мешков (a bag words model)

Разделяем все сообщения по типам личности и ***создаем новый фрейм данных для хранения всего этого в***

all_posts= pd.DataFrame()
for j in types:
    temp1 = data_set[data_set['type']==j]['posts']
    temp2 = []
    for i in temp1:
        temp2+=i.split('|||')
    temp3 = pd.Series(temp2)
    all_posts[j] = temp3

all_posts.tail()

### Создание функции для токенизации слов

useless_words = nltk.corpus.stopwords.words("english") + list(string.punctuation)
def build_bag_of_words_features_filtered(words):
    words = nltk.word_tokenize(words)
    return {
        word:1 for word in words \
        if not word in useless_words}

Рандомизированная проверка функции

build_bag_of_words_features_filtered(all_posts['INTJ'].iloc[1])

## Создание массива функций

features=[]
for j in types:
    temp1 = all_posts[j]
    temp1 = temp1.dropna() #not all the personality types have same number of files
    features += [[(build_bag_of_words_features_filtered(i), j) \
    for i in temp1]]

Поскольку у каждого типа личности разное количество постов, их нужно разделить соответственно. Взять 80% для обучения и 20% для тестирования

split=[]
for i in range(16):
    split += [len(features[i]) * 0.8]
split = np.array(split,dtype = int)

split

Данные для обучения

train=[]
for i in range(16):
    train += features[i][:split[i]] 

Обучение модели

sentiment_classifier = NaiveBayesClassifier.train(train)

Тестирование модели на датасете, на котором она была обучена, на предмет точности

nltk.classify.util.accuracy(sentiment_classifier, train)*100

Создание тестовых данных

test=[]
for i in range(16):
    test += features[i][split[i]:]

Тестирование модели на тестовом датасете, который она никогда раньше не видела

nltk.classify.util.accuracy(sentiment_classifier, test)*100

# Эффективность модели составляет всего 10%, что недостаточно.

## Поэтому вместо того, чтобы выбрать все 16 типов личностей в качестве уникальной характеристики, ислледуем датасет дальше и решаем упростить его до системы Кейрси.

Типология Майерс-Бриггс (MBTI) - это система типов личности, которая делит на 16 различных типов личности по психометрическим параметрам, используя 4 оси:

- Интроверсия (I) - Экстраверсия (E) - Introversion (I) – Extroversion (E) 
- Intuition (N) – Sensing (S) - Интуиция (N) - Чувствование (S)
- Thinking (T) – Feeling (F) - Мышление (T) - Чувство (F)
- Judging (J) – Perceiving (P) - Суждение (J) - Восприятие (P)
<br><br>
Кейрси предложил объеденить эти 4 дихотомии в 4 темперамента, воспользуемся его доработкой системы MBTI и продолжим:

## Создание классификатора для интроверсии (I) и экстраверсии (E)

**Note:** Детали шагов здесь такие же, как и при создании модели выше, поэтому тут только изменения

# Особенности модели the bag of words 
features=[]
for j in types:
    temp1 = all_posts[j]
    temp1 = temp1.dropna() #не все типы личности имеют одинаковое количество файлов
    if('I' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'introvert') \
        for i in temp1]]
    if('E' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'extrovert') \
        for i in temp1]]

Данные для обучения

train=[]
for i in range(16):
    train += features[i][:split[i]] 

Обучение модели

IntroExtro = NaiveBayesClassifier.train(train)

Тестирование модели на датасете, на котором она была обучена, на предмет точности

nltk.classify.util.accuracy(IntroExtro, train)*100

Создание тестовых данных

test=[]
for i in range(16):
    test += features[i][split[i]:]

Тестирование модели на тестовом датасете, который она никогда раньше не видела

nltk.classify.util.accuracy(IntroExtro, test)*100

Видим что эта модель дает неплохие результаты, повторим то же самое с остальными темпераментами

## Создание классификатора для интуиции (N) и сенсорики (S)

# Особенности модели the bag of words
features=[]
for j in types:
    temp1 = all_posts[j]
    temp1 = temp1.dropna() #не все типы личности имеют одинаковое количество файлов
    if('N' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'Intuition') \
        for i in temp1]]
    if('E' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'Sensing') \
        for i in temp1]]

Данные для обучения

train=[]
for i in range(16):
    train += features[i][:split[i]] 

Обучение модели

IntuitionSensing = NaiveBayesClassifier.train(train)

Тестирование модели на датасете, на котором она была обучена, на предмет точности

nltk.classify.util.accuracy(IntuitionSensing, train)*100

Создание тестовых данных

test=[]
for i in range(16):
    test += features[i][split[i]:]

Тестирование модели на тестовом датасете, который она никогда раньше не видела

nltk.classify.util.accuracy(IntuitionSensing, test)*100

## Создание классификатора для мышления (T) и чувств (F)

# Особенности модели  the bag of words
features=[]
for j in types:
    temp1 = all_posts[j]
    temp1 = temp1.dropna() #не все типы личности имеют одинаковое количество файлов
    if('T' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'Thinking') \
        for i in temp1]]
    if('F' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'Feeling') \
        for i in temp1]]

Данные для обучения

train=[]
for i in range(16):
    train += features[i][:split[i]] 

Обучение модели

ThinkingFeeling = NaiveBayesClassifier.train(train)

Тестирование модели на датасете, на котором она была обучена, на предмет точности

nltk.classify.util.accuracy(ThinkingFeeling, train)*100

Создание тестовых данных

test=[]
for i in range(16):
    test += features[i][split[i]:]

Тестирование модели на тестовом датасете, который она никогда раньше не видела

nltk.classify.util.accuracy(ThinkingFeeling, test)*100

## Создание классификатора для суждений (J) и восприятия (P)

# Особенности модели the bag of words 
features=[]
for j in types:
    temp1 = all_posts[j]
    temp1 = temp1.dropna() #не все типы личности имеют одинаковое количество файлов
    if('J' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'Judging') \
        for i in temp1]]
    if('P' in j):
        features += [[(build_bag_of_words_features_filtered(i), 'Percieving') \
        for i in temp1]]

Данные для обучения

train=[]
for i in range(16):
    train += features[i][:split[i]] 

Обучение модели

JudgingPercieiving = NaiveBayesClassifier.train(train)

Тестирование модели на датасете, на котором она была обучена, на предмет точности

nltk.classify.util.accuracy(JudgingPercieiving, train)*100

Создание тестовых данных

test=[]
for i in range(16):
    test += features[i][split[i]:]

Тестирование модели на тестовом датасете, который она никогда раньше не видела

nltk.classify.util.accuracy(JudgingPercieiving, test)*100

# Обобщение результатов работы моделей
***


temp = {'train' : [81.12443979837917,70.14524215640667,80.03456948570128,79.79341109742592], 'test' : [58.20469312585358,54.46262259027357,59.41315234035509,54.40549600629061]}
results = pd.DataFrame.from_dict(temp, orient='index', columns=['Introvert - Extrovert', 'Intuition - Sensing', 'Thinking - Feeling', 'Judging - Percieiving'])
results

Построим график результатов для большей наглядности

plt.figure(figsize = (12,6))

plt.bar(np.array(results.columns), height = results.loc['train'],)
plt.xlabel('Personality types', size = 14)
plt.ylabel('Number of posts available', size = 14)
plt.title('Total posts for each personality type')

labels = np.array(results.columns)

training = results.loc['train']
ind = np.arange(4)
width = 0.4
fig = plt.figure()
ax = fig.add_subplot(111)
rects1 = ax.bar(ind, training, width, color='royalblue')

testing = results.loc['test']
rects2 = ax.bar(ind+width, testing, width, color='seagreen')

fig.set_size_inches(12, 6)
fig.savefig('Results.png', dpi=200)

ax.set_xlabel('Model Classifying Trait', size = 18)
ax.set_ylabel('Accuracy Percent (%)', size = 18)
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(labels)
ax.legend((rects1[0], rects2[0]), ('Tested on a known dataframe', 'Tested on an unknown dataframe'))
plt.show()

# Тестирование моделей для предсказания черт характера по тексту из Quora

ссылка на ленту ответов quora: https://www.quora.com/profile/Divya-Bramhecha

Определение функции, которая вводит записи, обрабатывает их, а затем предсказывает результат на основе наших предыдущих классификаторов

def MBTI(input):
    tokenize = build_bag_of_words_features_filtered(input)
    ie = IntroExtro.classify(tokenize)
    Is = IntuitionSensing.classify(tokenize)
    tf = ThinkingFeeling.classify(tokenize)
    jp = JudgingPercieiving.classify(tokenize)
    
    mbt = ''
    
    if(ie == 'introvert'):
        mbt+='I'
    if(ie == 'extrovert'):
        mbt+='E'
    if(Is == 'Intuition'):
        mbt+='N'
    if(Is == 'Sensing'):
        mbt+='S'
    if(tf == 'Thinking'):
        mbt+='T'
    if(tf == 'Feeling'):
        mbt+='F'
    if(jp == 'Judging'):
        mbt+='J'
    if(jp == 'Percieving'):
        mbt+='P'
    return(mbt)
    

### Построение еще одной функции, которая принимает все сообщения в качестве входных данных и выводит график, показывающий процентное соотношение каждой черты, наблюдаемой в каждом сообщении, и суммирует, отображая вашу личность в качестве заголовка графика

**Note:**  В качестве входных данных должен быть массив постов

def tellmemyMBTI(input, name, traasits=[]):
    a = []
    trait1 = pd.DataFrame([0,0,0,0],['I','N','T','J'],['count'])
    trait2 = pd.DataFrame([0,0,0,0],['E','S','F','P'],['count'])
    for i in input:
        a += [MBTI(i)]
    for i in a:
        for j in ['I','N','T','J']:
            if(j in i):
                trait1.loc[j]+=1                
        for j in ['E','S','F','P']:
            if(j in i):
                trait2.loc[j]+=1 
    trait1 = trait1.T
    trait1 = trait1*100/len(input)
    trait2 = trait2.T
    trait2 = trait2*100/len(input)
    
    
    #Поиск психометрических параметров
    YourTrait = ''
    for i,j in zip(trait1,trait2):
        temp = max(trait1[i][0],trait2[j][0])
        if(trait1[i][0]==temp):
            YourTrait += i  
        if(trait2[j][0]==temp):
            YourTrait += j
    traasits +=[YourTrait] 
    
    #Начертание
    
    labels = np.array(results.columns)

    intj = trait1.loc['count']
    ind = np.arange(4)
    width = 0.4
    fig = plt.figure()
    ax = fig.add_subplot(111)
    rects1 = ax.bar(ind, intj, width, color='royalblue')

    esfp = trait2.loc['count']
    rects2 = ax.bar(ind+width, esfp, width, color='seagreen')

    fig.set_size_inches(10, 7)
    
    

    ax.set_xlabel('Finding the FFM Trait', size = 18)
    ax.set_ylabel('Trait Percent (%)', size = 18)
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(labels)
    ax.set_yticks(np.arange(0,105, step= 10))
    ax.set_title('Your Personality is '+YourTrait,size = 20)
    plt.grid(True)
    
    
    fig.savefig(name+'.png', dpi=200)
    
    plt.show()
    return(traasits)
        

# Импорт ответов на вопросы quora из текстового файла

Скопируем весь ответ из ссылки, которая была ранее (разбив на параграфы отдельные посты)

My_writings = open("Myquora.txt")
my_writing = My_writings.readlines()
#my_writing

my_posts = my_writing[0].split('|||')
len(my_posts)
#посты

# Использование классификатора для предсказания типа личности

trait=tellmemyMBTI(my_posts, 'Divy')


# Заключительная заметка

Профиль автора согласно https://www.16personalities.com/ это INTJ.

Используя эту базовую модель, она оказалась довольно близка к реальному профилю, всего 1 отличие. И даже это отличие незначительное, в пределах 10% неточностей, что довольно неплохо.

Хотя нет уверенности что классификатор будет работать на всех тестовых случаях т.к. данных и записей по некоторым профилям было очень мало.

# Проль Санайи

My_writings = open("Sanayapoem.txt")
my_writing = My_writings.readlines()
#my_writing

my_posts = my_writing[0].split('|||')
len(my_posts)
#my_posts

trait = tellmemyMBTI(my_posts,'sanaya')

# Валентин Пятаев

My_writings = open("Valentin pyatev.txt")
my_writing = My_writings.readlines()
#my_writing

my_posts = my_writing[0].split('|||')
len(my_posts)
#my_posts

trait=tellmemyMBTI(my_posts,'Valentin')

# Выборка людей

My_writings = open("All texts.txt")
my_writing = My_writings.readlines()
a =[''];
for i in my_writing:
    a[0]=a[0]+i
len(a)

my_posts = a[0].split('&&&')
len(my_posts)
#my_posts

Posts for each person

alls = [None]*len(my_posts)
for i in range(len(my_posts)):
    alls[i] = my_posts[i].split('|||') 

Email ID connection

Names = open("Names.txt")
names = Names.readlines()
#names

for i in range(len(names)):
    names[i] = names[i].replace('@gmail.com\n','')
    print(names[i])
names[len(names)-1]=names[len(names)-1].replace('@gmail.com','')

for i in range(len(alls)):
    trait=tellmemyMBTI(alls[i],names[i])

trait